In [34]:
!pip install torch torch_geometric

In [35]:
import torch
from torch_geometric.datasets import CoraFull
from torch_geometric.data import DataLoader

In [94]:
dataset = CoraFull(root="cora")
data = dataset[0]
len(data.x)

19793

In [38]:
print(len(data.x[0]))

8710


In [58]:
class MeanAggregator(torch.nn.Module):
    def __init__(self, in_features, out_features, neigh_input_dim):
        super(MeanAggregator, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.neigh_input_dim = neigh_input_dim

        self.w = torch.nn.Parameter(torch.empty(neigh_input_dim + in_features, out_features))
        torch.nn.init.xavier_uniform_(self.w)

    def forward(self, x, sampled_neighbors):
        neighbors_message = torch.mean(sampled_neighbors, dim=0)
        message = torch.concat((x, neighbors_message), 0)
        weighted_message = torch.matmul(message, self.w)
        return weighted_message

In [60]:
layer = MeanAggregator(len(data.x[0]), 70, len(data.x[0]))
node = data.x[0]
node_neighbors = torch.stack([data.x[v] for u, v in zip(data.edge_index[0], data.edge_index[1]) if data.edge_index[0][u] == 0])
print(layer(node, node_neighbors))

tensor([ 0.0273,  0.0101,  0.0834, -0.0051,  0.0301, -0.1831,  0.0112,  0.0334,
         0.0944, -0.0295,  0.0251,  0.0255,  0.0042, -0.0657, -0.0795, -0.1094,
        -0.1166, -0.0225,  0.0765,  0.0261, -0.0107,  0.0955, -0.0338,  0.0427,
         0.0034,  0.0728, -0.0336, -0.0131, -0.1184,  0.0072, -0.0156,  0.0417,
         0.0304, -0.1121,  0.0190,  0.0300,  0.0371, -0.0363, -0.1249,  0.0381,
         0.0892, -0.0598, -0.0583,  0.0055, -0.0929, -0.0621,  0.0336,  0.0311,
         0.0505,  0.0138,  0.0359,  0.0141, -0.1189,  0.1628, -0.1034,  0.0766,
         0.0239,  0.1485,  0.0019,  0.0027,  0.0208, -0.0792, -0.0252, -0.0816,
         0.0299, -0.0282, -0.0156,  0.1387,  0.0251,  0.0679],
       grad_fn=<SqueezeBackward4>)


In [ ]:
dataset.y[2272]

tensor(57)

In [82]:
class GraphSAGE(torch.nn.Module):
  def __init__(self):
    super(GraphSAGE, self).__init__()
    self.aggr1 = MeanAggregator(8710, 1000, 8710)
    self.aggr2 = MeanAggregator(1000, 70, 8710)

  def forward(self, input):
    x, neighbors = input
    h = torch.nn.functional.relu(self.aggr1(x, neighbors))
    logits = torch.nn.functional.relu(self.aggr2(h, neighbors))
    normalized = torch.nn.functional.softmax(logits)
    return torch.argmax(normalized)

In [83]:
model = GraphSAGE()
model((node, node_neighbors))

<ipython-input-82-07959d81957a>:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  normalized = torch.nn.functional.softmax(logits)


tensor(60)

In [95]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

epochs = 100
for epoch in range(epochs):
  for i in range(len(data.x)):
    x = data.x[i]
    node_neighbors = torch.stack([data.x[v] for u, v in zip(data.edge_index[0], data.edge_index[1]) if data.edge_index[i][u] == 0])
    y = data.y[i]

IndexError: index 2 is out of bounds for dimension 0 with size 2